# Chapter 14: Deep Computer Vision Using Convolutional Neural Networks

Bab ini menyelami dunia **Computer Vision** menggunakan **Convolutional Neural Networks (CNNs)**. CNNs adalah arsitektur jaringan saraf yang menjadi pendorong utama sebagian besar kemajuan dalam Computer Vision modern, mulai dari klasifikasi gambar, deteksi objek, hingga segmentasi gambar.

Arsitektur CNN terinspirasi oleh korteks visual otak hewan. Ide utamanya adalah bahwa neuron-neuron awal di korteks visual bereaksi terhadap area visual yang kecil dan tumpang tindih (disebut *receptive fields*). Pola ini diulang di seluruh bidang visual. Lapisan neuron yang lebih tinggi kemudian menggabungkan output dari lapisan bawahnya untuk mendeteksi pola yang lebih kompleks.

Kita akan membahas:
* Lapisan-lapisan fundamental yang membentuk sebuah CNN: **Convolutional Layer** dan **Pooling Layer**.
* Cara menumpuk lapisan-lapisan ini untuk membangun arsitektur CNN yang dalam.
* Mengimplementasikan CNN menggunakan Keras untuk tugas klasifikasi gambar.
* Arsitektur CNN modern yang canggih seperti **ResNet**, **Xception**, dan **SENet**.

## Convolutional Layers

**Convolutional Layer** adalah blok bangunan paling penting dari sebuah CNN. Alih-alih menghubungkan setiap neuron ke setiap piksel di input (seperti pada MLP), neuron di *convolutional layer* hanya terhubung ke piksel di *receptive field* lokalnya.

Lapisan ini bekerja dengan menggeser satu set bobot yang dapat dilatih, yang disebut **filter** (atau *kernel*), di seluruh gambar input. Setiap filter belajar untuk mendeteksi pola tertentu, seperti tepi vertikal, tepi horizontal, atau tekstur tertentu. Output dari lapisan ini adalah serangkaian **feature maps**, yang menunjukkan area di mana filter mendeteksi polanya.

Dua parameter penting dalam *convolutional layer* adalah:
* **`padding`**: Menambahkan piksel nol di sekitar batas gambar. Ini memungkinkan filter untuk diterapkan di tepi gambar dan mempertahankan ukuran spasial *feature map*.
* **`stride`**: Jarak geser filter di setiap langkah.

## Pooling Layers

Tujuan dari **Pooling Layer** adalah untuk melakukan *subsampling* (yaitu, mengecilkan) *feature map*. Ini membantu mengurangi jumlah parameter, beban komputasi, dan memori yang digunakan oleh jaringan, serta memberikan tingkat *invariance* (ketahanan) terhadap translasi kecil pada input.

Jenis *pooling* yang paling umum adalah **Max Pooling**. Ini bekerja dengan mengambil nilai maksimum dari setiap *region* kecil di *feature map*.

## Implementing a CNN with Keras

Arsitektur CNN yang umum biasanya terdiri dari beberapa *convolutional layer*, diikuti oleh *pooling layer*, diulang beberapa kali. Semakin dalam jaringannya, lapisan-lapisan awal cenderung belajar pola visual tingkat rendah (seperti tepi dan sudut), sementara lapisan yang lebih dalam belajar pola tingkat tinggi (seperti bentuk atau objek utuh). Di bagian akhir, *feature map* diratakan (*flattened*) dan dihubungkan ke beberapa *fully connected layer* (Dense) untuk menghasilkan prediksi akhir.

In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Memuat dataset Fashion MNIST
fashion_mnist = keras.datasets.fashion_mnist
(X_train_full, y_train_full), (X_test, y_test) = fashion_mnist.load_data()

# Membagi data dan melakukan penskalaan. Menambahkan dimensi channel.
X_train = X_train_full[5000:].astype(np.float32) / 255
X_valid = X_train_full[:5000].astype(np.float32) / 255
y_train = y_train_full[5000:]
y_valid = y_train_full[:5000]
X_train = X_train[..., np.newaxis]
X_valid = X_valid[..., np.newaxis]
X_test = X_test.astype(np.float32) / 255
X_test = X_test[..., np.newaxis]

# Membangun model CNN sederhana
model = keras.models.Sequential([
    keras.layers.Conv2D(64, 7, activation="relu", padding="same", input_shape=[28, 28, 1]),
    keras.layers.MaxPooling2D(2),
    keras.layers.Conv2D(128, 3, activation="relu", padding="same"),
    keras.layers.Conv2D(128, 3, activation="relu", padding="same"),
    keras.layers.MaxPooling2D(2),
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation="relu"),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(64, activation="relu"),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(10, activation="softmax")
])

model.compile(loss="sparse_categorical_crossentropy", optimizer="nadam", metrics=["accuracy"])
history = model.fit(X_train, y_train, epochs=10, validation_data=(X_valid, y_valid))

### Visualisasi Pelatihan

Setelah pelatihan, penting untuk memvisualisasikan kurva pembelajaran untuk melihat bagaimana kinerja model dari waktu ke waktu. Ini dapat membantu kita mendiagnosis masalah seperti *overfitting* atau *underfitting*.

In [ ]:
pd.DataFrame(history.history).plot(figsize=(8, 5))
plt.grid(True)
plt.gca().set_ylim(0, 1) # Mengatur batas sumbu y antara 0 dan 1
plt.title('Kurva Pembelajaran Model')
plt.xlabel('Epoch')
plt.ylabel('Akurasi / Loss')
plt.show()

## Modern CNN Architectures

Bidang Computer Vision telah berkembang pesat berkat inovasi dalam arsitektur CNN.

### ResNet (Residual Network)
Salah satu terobosan terbesar adalah **ResNet**, yang memperkenalkan konsep *residual connection* atau *skip connection*. Koneksi ini memungkinkan gradien untuk mengalir langsung melalui jaringan, melewati beberapa lapisan. Ini memungkinkan pelatihan jaringan yang sangat dalam (bahkan lebih dari 100 lapisan) tanpa mengalami masalah *vanishing gradient*.

### Xception & SENet
* **Xception:** Memperluas ide dari arsitektur Inception (GoogLeNet) dan menggabungkannya dengan *depthwise separable convolutions* untuk membuat arsitektur yang sangat efisien.
* **SENet (Squeeze-and-Excitation Network):** Meningkatkan arsitektur yang ada dengan menambahkan *SE blocks* yang secara eksplisit memodelkan ketergantungan antar-channel, memungkinkan jaringan untuk melakukan *feature recalibration* (belajar menyesuaikan bobot *feature map* secara adaptif).

In [ ]:
# Contoh konseptual sebuah Residual Block menggunakan Functional API
from tensorflow.keras.layers import Conv2D, Add, ReLU, Input
from tensorflow.keras.models import Model

inputs = Input(shape=(28, 28, 128))
conv1 = Conv2D(128, 3, padding='same')(inputs)
relu1 = ReLU()(conv1)
conv2 = Conv2D(128, 3, padding='same')(relu1)

# Skip Connection
skip_connection = Add()([inputs, conv2])
output = ReLU()(skip_connection)

residual_block = Model(inputs=inputs, outputs=output)
residual_block.summary()